In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

2021-11-28 20:40:43.339411: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-28 20:40:43.339438: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df=pd.read_csv("train_identity.csv").merge(pd.read_csv("train_transaction.csv"), how="right", left_on= "TransactionID", right_on= "TransactionID")

In [3]:
import pickle
with open('features_importantes', 'rb') as file:
    features_importantes = pickle.load(file)

In [4]:
del df["TransactionID"]

In [5]:
columnas=["TransactionDT", "isFraud"]
for c in features_importantes.index:
    columnas.append(df.columns[c])

In [6]:
df=df.filter(columnas)

In [7]:
for c in df.columns:
    if df[c].dtype==object: 
        df[c] = df[c].replace(np.nan, "NaN")
    else:
        df[c] = df[c].replace(np.nan, df[c].mean())

In [8]:
division_tran_validation = int(df["TransactionDT"][len(df)-1]*0.80)

In [9]:
df_train = df[df["TransactionDT"] <= division_tran_validation]
df_train_y = df_train["isFraud"]
df_validation = df[df["TransactionDT"] > division_tran_validation]
df_validation_y = df_validation["isFraud"]

In [10]:
%reset_selective -f "^df$"

In [11]:
lista_de_diccs = {}
one_hot_features = []
for c in df_train.columns:
    vectorizer = CountVectorizer()
    if df_train[c].dtype==object: 
        if len((df_train[c]).unique()) <= 10:
            one_hot = pd.DataFrame(vectorizer.fit_transform(df_train[c]).toarray()).rename(lambda x: c+sorted(vectorizer.vocabulary_ )[x], axis="columns")
            del df_train[c]
            one_hot_features.append(c)
            df_train = df_train.join(one_hot)
        else:
            dict_actual=df_train.groupby([c])['isFraud'].mean().to_dict()
            lista_de_diccs[c]=dict_actual
            df_train[c]=df_train[c].map(dict_actual)

In [12]:
for c in one_hot_features:
    one_hot = pd.DataFrame(vectorizer.transform(df_validation[c]).toarray()).rename(lambda x: c+sorted(vectorizer.vocabulary_ )[x], axis="columns").set_index(df_validation.index)
    df_validation = df_validation.join(one_hot)
    del df_validation[c]
for c in df_validation.columns:
    if df_validation[c].dtype==object: 
        df_validation[c]= df_validation[c].map(lista_de_diccs[c])
for c in df_validation.columns:
    df_validation[c] = df_validation[c].replace(np.nan, 0)

In [13]:
del df_train["isFraud"]
del df_validation["isFraud"]

In [14]:
del df_train["TransactionDT"]
del df_validation["TransactionDT"]

In [15]:
model = Sequential()

2021-11-28 20:41:05.117031: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-28 20:41:05.117056: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-28 20:41:05.117073: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bruno-VirtualBox): /proc/driver/nvidia/version does not exist
2021-11-28 20:41:05.118931: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
model.add(Dense(200, input_dim=100, activation="relu"))
model.add(Dense(200, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
training = model.fit(df_train, df_train_y, epochs=100, batch_size=64)

Epoch 1/100
7587/7587 [==============================] - 8s 994us/step - loss: 20.0111 - accuracy: 0.9428
Epoch 2/100
7587/7587 [==============================] - 7s 954us/step - loss: 0.1734 - accuracy: 0.9650
Epoch 3/100
7587/7587 [==============================] - 7s 955us/step - loss: 0.1495 - accuracy: 0.9664
Epoch 4/100
7587/7587 [==============================] - 7s 963us/step - loss: 0.1371 - accuracy: 0.9669
Epoch 5/100
7587/7587 [==============================] - 7s 955us/step - loss: 0.1343 - accuracy: 0.9672
Epoch 6/100
7587/7587 [==============================] - 7s 954us/step - loss: 0.1340 - accuracy: 0.9673
Epoch 7/100
7587/7587 [==============================] - 7s 961us/step - loss: 0.1320 - accuracy: 0.9671
Epoch 8/100
7587/7587 [==============================] - 7s 970us/step - loss: 0.1327 - accuracy: 0.9671
Epoch 9/100
7587/7587 [==============================] - 7s 963us/step - loss: 0.1323 - accuracy: 0.9672
Epoch 10/100
7587/7587 [==============================

Epoch 79/100
7587/7587 [==============================] - 7s 970us/step - loss: 0.1430 - accuracy: 0.9669
Epoch 80/100
7587/7587 [==============================] - 8s 996us/step - loss: 0.1460 - accuracy: 0.9667
Epoch 81/100
7587/7587 [==============================] - 8s 1ms/step - loss: 0.1369 - accuracy: 0.9669
Epoch 82/100
7587/7587 [==============================] - 8s 1ms/step - loss: 0.1311 - accuracy: 0.9674
Epoch 83/100
7587/7587 [==============================] - 8s 1ms/step - loss: 0.1289 - accuracy: 0.9676
Epoch 84/100
7587/7587 [==============================] - 8s 1ms/step - loss: 0.1277 - accuracy: 0.9678
Epoch 85/100
7587/7587 [==============================] - 8s 1ms/step - loss: 0.1281 - accuracy: 0.9680
Epoch 86/100
7587/7587 [==============================] - 8s 1ms/step - loss: 0.1265 - accuracy: 0.9682
Epoch 87/100
7587/7587 [==============================] - 8s 1ms/step - loss: 0.1266 - accuracy: 0.9684
Epoch 88/100
7587/7587 [==============================] - 8s

In [19]:
prediction=model.predict(df_validation)

In [20]:
roc_auc_score(df_validation_y, prediction)

0.7214465453981104

In [21]:
df_test = pd.read_csv("test_identity.csv").merge(pd.read_csv("test_transaction.csv"), how="right", left_on= "TransactionID", right_on= "TransactionID")

In [22]:
transactionid = df_test["TransactionID"]
for c in df_test.columns:
    df_test.rename(columns={c: c.replace("id-", "id_")}, inplace=True)

In [23]:
df_test=df_test.filter(columnas)

In [24]:
for c in df_test.columns:
    if df_test[c].dtype==object: 
        df_test[c] = df_test[c].replace(np.nan, "NaN")
    else:
        df_test[c] = df_test[c].replace(np.nan, df_test[c].mean())

In [25]:
for c in one_hot_features:
    one_hot = pd.DataFrame(vectorizer.fit_transform(df_test[c]).toarray()).rename(lambda x: c+sorted(vectorizer.vocabulary_ )[x], axis="columns")
    del df_test[c]
    df_test = df_test.join(one_hot)
for c in df_test.columns:
    if df_test[c].dtype==object: 
        df_test[c]= df_test[c].map(lista_de_diccs[c])

In [28]:
del df_test["TransactionDT"]

In [35]:
lista_de_probas = []
for i in list(model.predict(df_test)):
    lista_de_probas.append(i[0])

In [84]:
dataframe_final = pd.DataFrame({"TransactionID":transactionid, "isFraud":list(lista_de_probas)})

In [85]:
dataframe_final.to_csv("dataframe_entrega_extra.csv", index=False, na_rep = dataframe_final["isFraud"][0])

## Puntuacion Final del concurso para Keras
### 0.684401

### tuve que usar na_rep  porque por alguna razon to_csv me estaba borrando valores 